# ML Model persistence

## Train

https://scikit-learn.org/stable/modules/model_persistence.html

In [1]:
# irisSVM

from sklearn import svm
from sklearn import datasets
clf = svm.SVC()
X, y= datasets.load_iris(return_X_y=True)
clf.fit(X, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [4]:
X[0:1]

array([[5.1, 3.5, 1.4, 0.2]])

In [6]:
X[0:1].tolist()

[[5.1, 3.5, 1.4, 0.2]]

In [11]:
import json
json.dumps(X[0:1].tolist())

'[[5.1, 3.5, 1.4, 0.2]]'

## pickle
https://realpython.com/python-pickle-module/

In [8]:
import pickle
s = pickle.dumps(clf)

In [9]:
with open("irisSVM.pkl","wb") as f:
    f.write(s)

In [ ]:
clf2 = pickle.loads(s)
clf2.predict(X[0:1])

In [3]:
y[0]

0

In [5]:
from joblib import dump, load
dump(clf, 'irisSVM.joblib') 

['irisSVM.joblib']

In [ ]:
def index():
  payload = json.loads(request.get_data().decode('utf-8'))
  prediction = predict(payload['payload'])
  data = {}
  data['data'] = prediction[-1]
  return json.dumps(data)

In [10]:
import joblib
joblib.load('irisSVM.pkl')

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Flask

Test the API locally

`export FLASK_APP=predictions.py`

`flask run`

In [18]:
!export FLASK_APP=predictions.py

'export' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


## Zappa & Co

In [5]:
# Zappa requires a virtual environment
!python -m venv env

In [7]:
!pip install -r requirements.txt

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=fc9e8328a1759c7cc89ebdff63321655c70c3f21a0d095ee3dea69650d4ad7b9
  Stored in directory: c:\users\badmo\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


Activate the environment to run Zappa
`.\env\Scripts\activate`

In [15]:
# !zappa init

AWS Lambda requires your environment to have a maximum size of 50mb, but our packaged environment will be around 100mb. Lucky for us, it is possible for Lambda’s to load code from Amazon S3 without much performance loss (only a few milliseconds).

To activate this feature, you must add a new line to your [zappa_settings.json](zappa_settings.json)

`"slim_handler": true`

In [17]:
!zappa deploy dev

(pip 20.0.2 (c:\programdata\anaconda3\lib\site-packages), Requirement.parse('pip>=20.1'), {'pip-tools'})
Calling deploy for stage dev..
Error: Zappa requires an active virtual environment!
Learn more about virtual environments here: http://docs.python-guide.org/en/latest/dev/virtualenvs/


## S3 bucket

In [22]:
!aws s3 ls zappa-36d6b5iy7

2021-01-23 15:33:14     754785 dev_lambdaml_current_project.tar.gz


In [19]:
from boto.s3.key import Key
from boto.s3.connection import S3Connection

In [20]:
BUCKET_NAME = 'zappa-36d6b5iy7'

In [21]:
conn = S3Connection()
bucket = conn.create_bucket(BUCKET_NAME)

S3ResponseError: S3ResponseError: 400 Bad Request
<?xml version="1.0" encoding="UTF-8"?>
<Error><Code>InvalidRequest</Code><Message>The authorization mechanism you have provided is not supported. Please use AWS4-HMAC-SHA256.</Message><RequestId>DACD663C789B2AF7</RequestId><HostId>LHUoL3q5zOxmaB5MZ+zLeLG9ld080DhGC+gqf4USDdjA8ofSUrK4YGHOlFY2gBwBNxjUOCtDbzk=</HostId></Error>

In [ ]:
def load_model():
  conn = S3Connection()
  bucket = conn.create_bucket(BUCKET_NAME)
  key_obj = Key(bucket)
  key_obj.key = MODEL_FILE_NAME

  contents = key_obj.get_contents_to_filename(MODEL_LOCAL_PATH)
  return joblib.load(MODEL_LOCAL_PATH)

Delete bucket

In [24]:
!aws s3 rb s3://zappa-36d6b5iy7

remove_bucket failed: s3://zappa-36d6b5iy7 An error occurred (BucketNotEmpty) when calling the DeleteBucket operation: The bucket you tried to delete is not empty


In [25]:
!aws s3 ls s3://zappa-36d6b5iy7

2021-01-23 15:33:14     754785 dev_lambdaml_current_project.tar.gz


In [28]:
!aws s3 rm s3://zappa-36d6b5iy7/dev_lambdaml_current_project.tar.gz

delete: s3://zappa-36d6b5iy7/dev_lambdaml_current_project.tar.gz


In [29]:
!aws s3 rb s3://zappa-36d6b5iy7

remove_bucket: zappa-36d6b5iy7
